In [ ]:
!pip install torch
!pip install transformers

In [ ]:
!pip install diffgram
!pip install llama_index
!pip install matplotlib
!pip install tabulate

In [ ]:
import json
from diffgram import Project
from llama_index.core import SimpleDirectoryReader, StorageContext
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib as mpl
from tabulate import tabulate

In [ ]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader

In [ ]:
project_local = Project(host = "https://dd4d-2604-3d08-4f7f-e8c0-5844-ee59-358e-598.ngrok-free.app",
        project_string_id = "valiantbiter",
        client_id = "LIVE__ywmyayoir4c6zx1kz5j4",
        client_secret = "wwwh2eiwc7f4q35qywxgghuyfuh2oemwshec0tiu4gctapsxqe3orcyfe5u3")

## Create a directory in diffgram with dataset_name

In [ ]:
dataset_name = "dataset_from_api_v5"
#project_local.set_directory_by_name(dataset_name)
#directory = project_local.directory.new(name = dataset_name)

In [ ]:
## You may need to run this twice to see if the directory is created
directory = None
try:
    directory = project_local.directory.get(name = dataset_name)
except:
    directory = project_local.directory.new(name=dataset_name)
if directory is not None:    
    print(directory.__dict__)    

## Import all the files 
### make sure you have the diffgram_processing_v2 folder which has all the data arranged for NER task

In [ ]:
from llama_index.core import SimpleDirectoryReader, StorageContext

In [ ]:
file_metadata = lambda x: {"filename": x}
diffgram_documents = SimpleDirectoryReader("diffgram_processing_v2",file_metadata=file_metadata).load_data()

In [ ]:
print(len(diffgram_documents))

In [ ]:
print(diffgram_documents[0].get_text())

## Fetch the files and add it to diffgram
if you want to insert one file at a time
    file = project_local.file.from_local("diffgram_processing_v2/800.txt")

In [ ]:
minimum = min([len(diffgram_documents), 64])

In [ ]:
filename = diffgram_documents[1].metadata['filename']
file = project_local.file.from_local(filename,directory_id=directory.__dict__['id'])

In [ ]:
try:
    for document in range(0, minimum):
        filename = diffgram_documents[document].metadata['filename']
        file = project_local.file.from_local(filename,directory_id=directory.__dict__['id'])
        print(file)
        print(file.__dict__)
        print(dataset_name)
        file_list = []
        file_list.append(file.__dict__['id'])
        #result = directory.add(file_id_list = file_list)
        #print(result)
except:
    print("A file with this filename already exists")

## Fetch and add the schema

In [ ]:
f = open('legal_ner_schema.json')
NER_schema = json.load(f)

In [ ]:
## Id the Labels
# Create label to ID mapping
label2id = []
id2label = []
for id, label in enumerate(NER_schema['schema']):
    label2id.append(id)
    id2label.append(NER_schema['schema'][id]['name'])
print(label2id)
print(id2label)

In [ ]:
NER_schema_name = 'NER_TRAINING_SCHEMA'
schema_id = None
schemas = project_local.schema.list()
print(json.dumps(schemas, indent=2))
for schema in schemas:
    if schema['name'] == NER_schema_name:
        schema_id = schema['id']
if schema_id == None:
    json_response = project_local.new_schema(name=NER_schema_name)
    schema_id = json_response["id"]
    for NER in NER_schema['schema']:
        print(NER['name'])
        project_local.label_new(NER, schema_id=schema_id)

## Create a Task 

In [ ]:
member_list = project_local.get_member_list()
member_list_ids = [x['member_id'] for x in member_list]

In [ ]:
schemas = project_local.schema.list()
print(json.dumps(schemas, indent=2))

In [ ]:
JOB_NAME = "Law_NER_task1"

In [ ]:
job = project_local.job.new(
    name = JOB_NAME,
    instance_type = "box",
    share = "Project",
    sync_directories = [directory],
    label_schema_id = schema_id,
    tag_list = ["Laws", "Acts", "Regulations"],
    members_list_ids = member_list_ids,
    auto_launch = True
)

## Export annotated data

In [ ]:
## Get the job based on the id and name
def get_completed_job():
    get_job = project_local.job.list()
    for jobs_completed in get_job:
        if (jobs_completed['status'] == 'complete') and (jobs_completed['name'] == JOB_NAME):
            return jobs_completed
        else:
            continue

In [ ]:
completed_job = get_completed_job()
if completed_job is not None:
    print(f"The status of the job {completed_job['name']} (id = {completed_job['id']})  is {completed_job['status']}.")

In [ ]:
results = project_local.job

In [ ]:
results.refresh_from_dict(completed_job)

In [ ]:
completed_annotations = results.generate_export()

In [ ]:
DIFFGRAM_EXPORT_ADDITIONAL_PARAMETERS = 5

In [ ]:
print(f" Number of annotated data {len(completed_annotations) - DIFFGRAM_EXPORT_ADDITIONAL_PARAMETERS}")

## Post Processing
### processing the annotated data to train the A.I

In [ ]:
data = []
sentences = []
labels = []
data_index = 0
for completed_annotation in completed_annotations:
    #print(f"{completed_annotation} ----")
    if (completed_annotation != 'attribute_groups_reference')  \
        and (completed_annotation != 'export_info') \
        and (completed_annotation != 'label_map') \
        and (completed_annotation != 'readme') \
        and (completed_annotation != 'label_colour_map'):
        sentence_local = []
        labels_local = []
        # First get the point where the annotation is started
        for start in completed_annotations[completed_annotation]['instance_list']:
            if 'start_token' in start:
                start_token =  start['start_token']
                break
            
        #start_token = completed_annotations[completed_annotation]['instance_list'][0]['start_token']
        for annotated_index in range(start_token, len(completed_annotations[completed_annotation]['text']['tokens']['words'])):
            # check if this text is annotated
            for data in completed_annotations[completed_annotation]['instance_list']:
                if 'start_token' in data:
                    if annotated_index == data['start_token']:
                        sentence_local.append(completed_annotations[completed_annotation]['text']['tokens']['words'][annotated_index]['value'])
                        labels_local.append(completed_annotations['label_map'][str(data['label_file_id'])])
                        #print(f"{completed_annotations[completed_annotation]['text']['tokens']['words'][annotated_index]['value']} - {completed_annotations['label_map'][str(data['label_file_id'])]}")
                        break;
        sentences.append(sentence_local)       
        labels.append(labels_local)
        data_index+=1

In [ ]:
len(sentences)

In [ ]:
# Create DataFrame
# Creating a pandas DataFrame
for iter in range (0, len(sentences)):
    pd.set_option('display.max_colwidth', None)
    ner_df = pd.DataFrame({
        "Token": sentences[iter],
        "Label": labels[iter]
    })  
    # Display the DataFrame with some custom styles
    ner_df.style.set_properties(**{'background-color': 'lightyellow', 
                               'color': 'black',
                               'border-color': 'black'})
    # Use tabulate to display a nice table in the terminal
    print(tabulate(ner_df, headers='keys', tablefmt='fancy_grid'))

## Preparing the data for input to pytorch
### We need to make all the data the same length by padding if they are not 255 tokens in length

In [ ]:
# Load pre-trained tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
MAX_LENGTH = 255
PAD_TOKEN = "[PAD]"
PAD_LABEL = "O"

In [ ]:
def pad_sequence(sequence, max_length, pad_value):
    return sequence + [pad_value] * (max_length - len(sequence))

In [ ]:
# Create DataFrame
data = []
NER_label = []
for sent_id, (sentence, sent_labels) in enumerate(zip(sentences, labels)):
    tmp_data = []
    tokens = sentence  # Simple tokenization
    #print(tokens)
    sentence_string = ""
    for string_value in tokens:
        sentence_string += string_value + ' '
    #print(sentence_string)
    padded_tokens = pad_sequence(tokens, MAX_LENGTH, PAD_TOKEN)
    padded_labels = pad_sequence(sent_labels, MAX_LENGTH, PAD_LABEL)
    encoded = tokenizer(sentence_string,
                        padding='max_length',
                        truncation=True,
                        max_length=MAX_LENGTH,  # Adjust based on your needs
                        return_tensors='pt')
    #print(encoded['input_ids'][0][0])
    data.append(encoded)
    NER_label.append(padded_labels)
    for token_id in range(MAX_LENGTH):
        tmp_data.append({
            'Sentence_ID': sent_id,
            'Token_ID': token_id,
            'Token': padded_tokens[token_id],
            'NER_Label': padded_labels[token_id],
            'Is_Pad': padded_tokens[token_id] == PAD_TOKEN
        })
    #data.append(tmp_data)

In [ ]:
def convert_label_to_id(ner_label,id2label):
    label_id = []
    for label in ner_label:
        label_id.append(id2label.index(label))
    return label_id

## Training the model

In [ ]:
# Load pre-trained model
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id)

# Training loop (simplified)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:
device = "cpu"

In [ ]:
for epoch in range(3):  # Number of epochs
    for i, batch in enumerate(data):
        labelled_id = convert_label_to_id(NER_label[i], id2label)
        print(labelled_id)
        input_ids = torch.tensor(batch['input_ids']).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).to(device)
        labels = torch.tensor(labelled_id, dtype=torch.long).to(device)

        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"Epoch: {epoch}, Loss: {loss.item()}")

## Evaluating the model

In [ ]:
model.eval()

In [ ]:
test_data = data
with torch.no_grad():
    for i, batch in enumerate(test_data):
        # Convert inputs to tensors
        input_ids = torch.tensor(batch['input_ids']).to(device)
        attention_mask = torch.tensor(batch['attention_mask']).to(device)
        
        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        print(predictions)
        
        # Compare predictions with true labels if available
        #true_labels = batch['labels']  # Assuming test data has true labels
        # Calculate accuracy or other metrics

# ------------------------------------------------------------------------------------------------
# **************************************  End of Script  *****************************************
# ------------------------------------------------------------------------------------------------